## A simple test on parallel run module
This nobebook sample require you have complete the "Setup environment" part in [azureml-module-get-started.ipynb](https://github.com/lisagreenview/hello-aml-modules/blob/master/azureml-module-get-started.ipynb) From this example, you will know:

1. How to prepare a parallel run module
2. How to use additional include feature
3. How to use .amlignore when register module
4. How to set run setting for parallel run module in pipeline run

In [ ]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
workspace_name = 'lisal-amlservice'
resource_group = 'lisal-dev'

# Specify available aml compute in workspace
pipeline_compute = "always-on-ds2v2"

In [ ]:
# Register module from github
!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/sc_tokenizer/module_spec.yaml

# Or register module from local
# set defaule amlignore
# !az configure --defaults module_amlignore_file=./.amlignore
# !az ml module register --spec-file=./sc_tokenizer/module_spec.yaml

In [ ]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module, dsl

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

In [ ]:
# get module func from registered module
tokenize_func = Module.load(ws, namespace='microsoft.com/aml/sc', name='Tokenizer')

# or get module func from anonymous module
# tokenize_func = Module.load_from_yaml(ws, yaml_file='./sc_tokenizer/module_spec.yaml')

In [ ]:
data_name = 'Company'
input_data = ws.datasets[data_name]

In [ ]:
@dsl.pipeline(name = 'Tokenizer - parallel run', 
              description = 'a pipeline to test parallel run',
              default_compute_target = pipeline_compute)
def tokenize_pipeline():
    tokenizer = tokenize_func(input_file_path = input_data)
    tokenizer.runsettings.configure(node_count=2,process_count_per_node=1)

    return {**tokenizer.outputs}

# create a pipeline
pipeline = tokenize_pipeline()

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
pipeline.save(experiment_name = 'Tokenizer-sdk')

In [ ]:
run = pipeline.submit(experiment_name = 'Tokenizer-sdk')
run.wait_for_completion()